In [0]:
#McEliece dengan kode golay

In [2]:
from sage.all import *
import random
import numpy as np
from sage.crypto.util import bin_to_ascii,ascii_to_bin


def error_vector(n,t):
    error_vector = vector(GF(2), n)
    points = random.sample(range(n), t)
    for i in range(t):
        error_vector[points[i]] += 1
    return error_vector
def non_singular_matrix(F, k):
    while 1:
        try:
            r = random_matrix(F, k, k)
            if r.inverse() != ZeroDivisionError:
                break
        except ZeroDivisionError:
            continue
    return r
def rand_perm(F, n):
    r = matrix(F, n, n)
    points = random.sample(range(n), n)
    for i in range(n):
        r[i, points[i]] = 1
    v = vector(F, n)
    for i in range(n):
        v += r[i]
    assert v + v == 0
    return r
def generate_key(priv_key_file,pub_key_file):
    golay = codes.GolayCode(GF(2),extended=True)
    g = golay.generator_matrix()
    h = golay.parity_check_matrix()
    s = non_singular_matrix(GF(2),12)
    p = rand_perm(GF(2), 24)
    p1 = p.inverse()
    s1 = s.inverse()
    g1 = s*g*p
    np.savez_compressed(priv_key_file, s=s,p=p, p1 = p1, s1 = s1)
    print('priv key berhasil disimpan')
    np.savez_compressed(pub_key_file, g1=g1)
    print('pub key berhasil disimpan')
def padder(bitstream, wordlength=12):
    prefix, padding = '',''
    length = len(bitstream)
    remainder = length % wordlength
    if remainder != 0:
        padlength = wordlength - remainder
        prefix = '{0:012b}'.format(padlength)
        padding = '1' * padlength
    return prefix + bitstream + padding
def unpadder(bitstream, wordlength=12):
    padlength = int(bitstream[:wordlength],2)
    return bitstream[wordlength:-padlength]
def encode(msg,pk):
    e = error_vector(24,3)
    c = msg*pk+e
    return c
def decode(c,s1,p1):
    y = c*p1
    d = codes.GolayCode(GF(2),extended=True).decoder()
    ms = d.decode_to_message(y)
    mor = ms*s1
    return mor
def encrypt(file_name,pub_key_file):
    filename = file_name
    key = np.load(pub_key_file, allow_pickle=True)
    pk = matrix(GF(2), key['g1'])
    with open(filename,"rb") as fh:
        bytestream = fh.read()
        bytestream = bytearray(bytestream)
    blok = 0
    encodeds = []
    for i, b in enumerate(bytestream):
        binary = '{0:08b}'.format(b)
        binary = padder(binary)
        encoded = ''
        for index in range(0, len(binary), 12):
            word = binary[index:index+12]
            f = vector(GF(2),word)
            en = encode(f,pk)
            for i in en:
                encoded += str(i)
            blok += 1
        for index in range(0, len(encoded), 8):
            word = encoded[index:index+8]
            word = int(word,2)
            encodeds.append(word)
        print("{0} block telah dienkripsi".format(blok))
    encodeds = bytearray(encodeds)
    fh = open("enc_"+filename,"wb")
    fh.write(encodeds)
    fh.close()
    print(encodeds)
    print("enkripsi selesai")
def decrypt(file_name,priv_key_file):
    filename = file_name
    key = np.load(priv_key_file,allow_pickle=True)
    S = matrix(GF(2),key['s'])
    P = matrix(GF(2),key['p'])
    Sinv = matrix(GF(2),key['s1'])
    Pinv = matrix(GF(2),key['p1'])
    with open(filename,"rb") as fh:
        bytestream = fh.read()
        bytestream = bytearray(bytestream)
    encoded = ''
    decodeds = []
    for i, b in enumerate(bytestream):
        binary = '{0:08b}'.format(b)
        encoded += binary
    blok = 0
    decodeds = ''
    decoded = ''
    for index in range(0,len(encoded),24):
        word = encoded[index:index+24]
        f = vector(GF(2),word)
        de = decode(f,Sinv,Pinv)
        for i in de:
            decoded += str(i)
        blok+=1
        print("{0} block dari {1} block".format(blok,int(len(encoded)/24)))
    for index in range(0,len(decoded),24):
        word = decoded[index:index+24]
        word = unpadder(word)
        decodeds+= word
    decoded_list = []
    for index in range(0, len(decodeds), 8):
        word = decodeds[index:index+8]
        word = int(word,2)
        decoded_list.append(word)
    decoded_list = bytearray(decoded_list)
    fh = open("dec_"+filename,"wb")
    fh.write(decoded_list)
    fh.close()
    print("dekripsi selesai")


In [2]:
with open("x.png","rb") as fh:
    bytestream = fh.read()
    bitstream = ''
    for i, b in enumerate(bytestream):
        binary = '{0:08b}'.format(b)
        bitstream += binary
    print(len(bytestream))

195


In [3]:
#enkripsi
filename = "x.png"
key = np.load("public.npz", allow_pickle=True)

pk = matrix(GF(2), key['g1'])
with open(filename,"rb") as fh:
    bytestream = fh.read()
    bytestream = bytearray(bytestream)
blok = 0
encodeds = []
for i, b in enumerate(bytestream):
    binary = '{0:08b}'.format(b)
    binary = padder(binary)
    encoded = ''
    for index in range(0, len(binary), 12):
        word = binary[index:index+12]
        f = vector(GF(2),word)
        en = encode(f,pk)
        for i in en:
            encoded += str(i)
        blok += 1
    for index in range(0, len(encoded), 8):
        word = encoded[index:index+8]
        word = int(word,2)
        encodeds.append(word)
    print("{0} block telah dienkripsi".format(blok))
encodeds = bytearray(encodeds)
fh = open("enc_"+filename,"wb")
fh.write(encodeds)
fh.close()
print(encodeds)
print("enkripsi selesai")

2 block telah dienkripsi
4 block telah dienkripsi
6 block telah dienkripsi
8 block telah dienkripsi
10 block telah dienkripsi
12 block telah dienkripsi
14 block telah dienkripsi
16 block telah dienkripsi
18 block telah dienkripsi
20 block telah dienkripsi
22 block telah dienkripsi
24 block telah dienkripsi
26 block telah dienkripsi
28 block telah dienkripsi
30 block telah dienkripsi
32 block telah dienkripsi
34 block telah dienkripsi
36 block telah dienkripsi
38 block telah dienkripsi
40 block telah dienkripsi
42 block telah dienkripsi
44 block telah dienkripsi
46 block telah dienkripsi
48 block telah dienkripsi
50 block telah dienkripsi
52 block telah dienkripsi
54 block telah dienkripsi
56 block telah dienkripsi
58 block telah dienkripsi
60 block telah dienkripsi
62 block telah dienkripsi
64 block telah dienkripsi
66 block telah dienkripsi
68 block telah dienkripsi
70 block telah dienkripsi
72 block telah dienkripsi
74 block telah dienkripsi
76 block telah dienkripsi
78 block telah d

272 block telah dienkripsi
274 block telah dienkripsi
276 block telah dienkripsi
278 block telah dienkripsi
280 block telah dienkripsi
282 block telah dienkripsi
284 block telah dienkripsi
286 block telah dienkripsi
288 block telah dienkripsi
290 block telah dienkripsi
292 block telah dienkripsi
294 block telah dienkripsi
296 block telah dienkripsi
298 block telah dienkripsi
300 block telah dienkripsi
302 block telah dienkripsi
304 block telah dienkripsi
306 block telah dienkripsi
308 block telah dienkripsi
310 block telah dienkripsi
312 block telah dienkripsi
314 block telah dienkripsi
316 block telah dienkripsi
318 block telah dienkripsi
320 block telah dienkripsi
322 block telah dienkripsi
324 block telah dienkripsi
326 block telah dienkripsi
328 block telah dienkripsi
330 block telah dienkripsi
332 block telah dienkripsi
334 block telah dienkripsi
336 block telah dienkripsi
338 block telah dienkripsi
340 block telah dienkripsi
342 block telah dienkripsi
344 block telah dienkripsi
3

In [3]:
#dekripsi
filename = "enc_x.png"
key = np.load("private.npz",allow_pickle=True)

S = matrix(GF(2),key['s'])
P = matrix(GF(2),key['p'])
Sinv = matrix(GF(2),key['s1'])
Pinv = matrix(GF(2),key['p1'])
with open(filename,"rb") as fh:
    bytestream = fh.read()
    bytestream = bytearray(bytestream)
encoded = ''
decodeds = []
for i, b in enumerate(bytestream):
    binary = '{0:08b}'.format(b)
    encoded += binary
blok = 0
decodeds = ''
decoded = ''
for index in range(0,len(encoded),24):
    word = encoded[index:index+24]
    f = vector(GF(2),word)
    de = decode(f,Sinv,Pinv)
    for i in de:
        decoded += str(i)
    blok+=1
    print("{0} block dari {1} block".format(blok,int(len(encoded)/24)))
for index in range(0,len(decoded),24):
    word = decoded[index:index+24]
    word = unpadder(word)
    decodeds+= word
decoded_list = []
for index in range(0, len(decodeds), 8):
    word = decodeds[index:index+8]
    word = int(word,2)
    decoded_list.append(word)
decoded_list = bytearray(decoded_list)
fh = open("dec_"+filename,"wb")
fh.write(decoded_list)
fh.close()
print("dekripsi selesai")

1 block dari 390 block


2 block dari 390 block


3 block dari 390 block


4 block dari 390 block


5 block dari 390 block


...

386 block dari 390 block


387 block dari 390 block


388 block dari 390 block


389 block dari 390 block


390 block dari 390 block
dekripsi selesai


In [30]:
generate_key("privat","publik")

priv key berhasil disimpan
pub key berhasil disimpan


In [3]:
with open("dec_enc_contoh","rb") as fh:
    bytestream = fh.read()
    print(bytestream)

b'Matematika 2019'


In [32]:
encrypt("contoh","publik.npz")

2 block telah dienkripsi
4 block telah dienkripsi
6 block telah dienkripsi
8 block telah dienkripsi
10 block telah dienkripsi
12 block telah dienkripsi
14 block telah dienkripsi
16 block telah dienkripsi
18 block telah dienkripsi
20 block telah dienkripsi
22 block telah dienkripsi
24 block telah dienkripsi
26 block telah dienkripsi
28 block telah dienkripsi
30 block telah dienkripsi
bytearray(b'u\xc5G\xe9\x0e\xe6u\xc3G\xbf\x8c\xd6G\xc1A\x939\xc2oQG\xe9A\xf3\xef\xc1FI`xg\x91O\xb2\xc4\xdeg\xf1\xc7\xb3\xb9Fe\xc1\xc3W\xbdZw\xe1F\t\x08$fAO\xb6\x0e\xd6u\xc1W\xa0p\xe7e\xc8G_\xbeOc\xc0E`\x9f\xbf\xf7\xc1EV\x80\xc6e\xc1w\xc2\x01\x1a')
enkripsi selesai


In [33]:
decrypt("enc_contoh","privat.npz")

1 block dari 30 block


2 block dari 30 block


3 block dari 30 block


4 block dari 30 block


5 block dari 30 block


...

26 block dari 30 block


27 block dari 30 block


28 block dari 30 block


29 block dari 30 block


30 block dari 30 block
dekripsi selesai


In [29]:
key = np.load("privat.npz",allow_pickle=True)
Sinv = matrix(GF(2),key['s1'])
Pinv = matrix(GF(2),key['p1'])
print(Sinv)

BadZipFile: Bad offset for central directory